In [1]:
import numpy as np
import pandas as pd

In [2]:
from irec.environment.loader import FullData

In [3]:
from irec.recommendation.agents import SimpleAgent
from irec.recommendation.agents.action_selection_policies import ASPEGreedy, ASPGreedy

In [4]:
from irec.recommendation.agents.value_functions import *
# from irec.recommendation.agents.value_functions.nicf import NICF

In [5]:
import sys
sys.path.append("..")
from wscb.value_function import WSCB

In [6]:
from irec.offline_experiments.evaluation_policies import FixedInteraction
from irec.offline_experiments.metric_evaluators import UserCumulativeInteraction, StageIterations
from irec.offline_experiments.metrics import Hits

In [7]:
# Dataset
dataset = {
    'path': "../dataset/MovieLens 100k/ratings.csv",
    'random_seed': 0,
    'file_delimiter': ",",
    'skip_head': True
}
# Splitting
splitting = {'strategy': "temporal", 'train_size': 0.8, 'test_consumes': 5}
# Loader
loader = FullData(dataset, splitting)
train_dataset, test_dataset, _, _ = loader.process()


Applying splitting strategy: temporal

Test shape: (16892, 4)
Train shape: (83108, 4)


In [8]:
parameters = {
    "Random": {},
    "MostPopular": {},
    "UCB": {"c": 0.25},
    "ThompsonSampling": {"alpha_0": 1, "beta_0": 100},
    "EGreedy": {"epsilon": 0.1},
    "LinearUCB": {"alpha": 1.0, "item_var": 0.01, "iterations": 20, "num_lat": 20, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    "GLM_UCB": {"c": 4, "item_var": 0.01, "iterations": 20, "num_lat": 10, "stop_criteria": 0.0009, "user_var": 0.01, "var": 0.05},
    # "NICF": {"batch": 256, "clip_param": 0.2, "dropout_rate": 0.01, "gamma": 0.0, "inner_epoch": 200, "latent_factor": 10, "learning_rate": 0.001, "num_blocks": 1, "num_heads": 2, "restore_model": False, "rnn_layer": 1, "time_step": 100, "training_epoch": 10000},
    "PTS": {"num_lat": 20, "num_particles": 5, "var": 0.5, "var_u": 1.0, "var_v": 1.0},
    "ICTRTS": {"num_lat": 2, "num_particles": 5},
    "ClusterBandit": {"B": 5, "C": 0.5, "D": 3, "num_clusters": 4, "num_lat": 10},
    "WSCB": {"alpha": 0.75, "num_lat": 10}
}

In [9]:
agents = {}
for agent_name, prmt in parameters.items():
    value_function = eval(agent_name)(**prmt)
    action_selection_policy = ASPEGreedy(**prmt) if agent_name == "EGreedy" else ASPGreedy()
    agent = SimpleAgent(value_function, action_selection_policy, name=agent_name)
    agents[agent_name] = agent

In [10]:
eval_policy = FixedInteraction(num_interactions=100, interaction_size=1, save_info=False)

In [11]:
interactions = {}
for agent_name, agent in agents.items():
    print(agent_name)
    agent_interactions, action_info = eval_policy.evaluate(agent, train_dataset, test_dataset)
    interactions[agent_name] = agent_interactions

Random
Starting Random Training
Ended Random Training


Random: 100%|██████████| 18900/18900 [00:02<00:00, 9426.73it/s] 


MostPopular
Starting MostPopular Training
Ended MostPopular Training


MostPopular: 100%|██████████| 18900/18900 [00:02<00:00, 8971.85it/s]


UCB
Starting UCB Training
Ended UCB Training


...

ICTRTS: 100%|██████████| 18900/18900 [05:25<00:00, 58.04it/s]


ClusterBandit
Starting ClusterBandit Training
Ended ClusterBandit Training


ClusterBandit: 100%|██████████| 18900/18900 [01:23<00:00, 227.04it/s] 


WSCB
Starting WSCB Training
0.0 1.0
Ended WSCB Training


WSCB: 100%|██████████| 18900/18900 [00:14<00:00, 1287.75it/s]


In [12]:
# Cumulative Evaluation Setup
evaluator = UserCumulativeInteraction(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [13]:
# Getting the results
cumulative_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    cumulative_results[agent_name] = hits_values


Evaluating Random

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 1.22 seconds executing Hits metric

Evaluating MostPopular

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction 100 with UserCumulativeInteraction
UserCumulativeInteraction spent 0.75 seconds executing Hits metric

Evaluating UCB

Computing interaction 5 with UserCumulativeInteraction
Computing interaction 10 with UserCumulativeInteraction
Computing interaction 20 with UserCumulativeInteraction
Computing interaction 50 with UserCumulativeInteraction
Computing interaction

In [14]:
df_cumulative = pd.DataFrame(columns=["Model", 5, 10, 20, 50, 100])
df_cumulative.Model = list(cumulative_results.keys())
df_cumulative.set_index("Model", inplace=True)
for agent_name, results in cumulative_results.items():
    df_cumulative.loc[agent_name] = [
        np.mean(list(metric_values.values())) for metric_values in results
    ]
df_cumulative

,5,10,20,50,100
Model,,,,,
Random,0.174603,0.37037,0.624339,1.529101,2.984127
MostPopular,1.666667,2.915344,5.137566,10.37037,16.873016
UCB,1.074074,2.074074,3.772487,8.275132,14.005291
ThompsonSampling,1.619048,2.94709,5.132275,10.417989,17.513228
EGreedy,1.047619,2.0,3.724868,8.095238,13.825397
LinearUCB,1.878307,3.566138,6.529101,14.074074,22.936508
GLM_UCB,1.232804,2.148148,5.100529,12.116402,20.941799
PTS,0.412698,0.698413,1.248677,4.407407,11.428571
ICTRTS,0.084656,0.26455,0.883598,4.216931,9.529101


In [15]:
# Stage Evaluation Setup
evaluator = StageIterations(
    ground_truth_dataset=test_dataset,
    num_interactions=100,
    interaction_size=1,
    interactions_to_evaluate=[5, 10, 15, 20, 50, 100],
    relevance_evaluator_threshold=3.99
)

In [16]:
# Getting the results
stage_results = {}
for agent_name, agent_results in interactions.items():
    print(f"\nEvaluating {agent_name}\n")
    hits_values = evaluator.evaluate(metric_class=Hits, results=agent_results)
    stage_results[agent_name] = hits_values


Evaluating Random

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 15 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.93 seconds executing Hits metric

Evaluating MostPopular

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 15 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.46 seconds executing Hits metric

Evaluating UCB

Computing iteration 5 with StageIterations
Computing iteration 10 with StageIterations
Computing iteration 15 with StageIterations
Computing iteration 20 with StageIterations
Computing iteration 50 with StageIterations
Computing iteration 100 with StageIterations
StageIterations spent 0.39 secon

In [17]:
df_stage = pd.DataFrame(columns=["Model", "1-5", "6-10", "11-15", "16-20", "21-50", "51-100"])
df_stage.Model = list(stage_results.keys())
df_stage.set_index("Model", inplace=True)
for agent_name, results in stage_results.items():
    df_stage.loc[agent_name] = [
        np.mean(list(metric_values.values())) for metric_values in results
    ]
df_stage

,1-5,6-10,11-15,16-20,21-50,51-100
Model,,,,,,
Random,0.174603,0.195767,0.111111,0.142857,0.904762,1.455026
MostPopular,1.666667,1.248677,1.248677,0.973545,5.232804,6.502646
UCB,1.074074,1.0,0.910053,0.78836,4.502646,5.730159
ThompsonSampling,1.619048,1.328042,1.132275,1.05291,5.285714,7.095238
EGreedy,1.047619,0.952381,0.920635,0.804233,4.37037,5.730159
LinearUCB,1.878307,1.687831,1.560847,1.402116,7.544974,8.862434
GLM_UCB,1.232804,0.915344,1.470899,1.481481,7.015873,8.825397
PTS,0.412698,0.285714,0.269841,0.280423,3.15873,7.021164
ICTRTS,0.084656,0.179894,0.238095,0.380952,3.333333,5.312169
